# Log into an Existing FABRIC node

This notebook will show you how to get an existing slice, get a node from the slice, get the IP and make the ssh command that you can use to access the node.


## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
bastion_public_addr = 'bastion-1.fabric-testbed.net'
bastion_private_ipv4_addr = '192.168.11.226'
bastion_private_ipv6_addr = '2600:2701:5000:a902::c'

bastion_username = '<your bastion username>'

bastion_key_filename = os.environ['HOME'] + "/.ssh/id_rsa_fabric"

ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

## Create slice manager object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Configure Slice Parameters



In [ ]:
slice_name="MySlice"
node_name="Node1"
username='ubuntu' # or 'centos'

## Query Slices

You can get a list of all your slices from the slice manager. If this is your first slice, it should return only one slice.

Note that the status returned by the call to slices indicates the success or failure of the call to the FABRIC control framework. The status is not the status of the slices.  The status of each slice is included in the list of slices.

In [ ]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

## Get the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

In [ ]:
slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]

print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

## Get the Slice Topology

In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

## Get the Node

In [ ]:
node = experiment_topology.nodes[node_name]
management_ip = str(node.get_property(pname='management_ip'))
print("Node:")
print("   Management IP     : {}".format(node.get_property(pname='management_ip')))
print()    

## Create the ssh command

We have the management_ip. Now we print the SSH command that we can use to access the machine. We need to use the key path variable that we've defined at the time of slice creation. We have the path to our key stored in the "ssh_key_file_priv" variable.

The result of the command produced by the cell can be use a terminal to access the VM.

In [ ]:
print("ssh -i {} -J {}@{} -i {} {}@{}".format(bastion_key_filename,
                                              bastion_username,
                                              bastion_public_addr,
                                              ssh_key_file_priv,
                                              username,
                                              management_ip))